In [1]:
import time 
import multiprocessing as mp
from multiprocessing import Pool
from collections import defaultdict
import sys
sys.path.append('/home/ak/Documents/PaperCode/stylised_facts')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import lob_for_futures as lobFut
from lob_for_futures import *

from scipy.stats import jarque_bera
import matplotlib.pyplot as plt

from dateutil.parser import parse
from pandas.tseries.offsets import BDay
import pickle as pkl
import fnmatch
import pickle
import datetime as dt
from datetime import timedelta
import glob
import fathon
from fathon import fathonUtils as fu
import itertools
import pyinform as pyinf
# import pingouin as pig
import seaborn as sns
from scipy.stats import entropy
from scipy.spatial.distance import jensenshannon
from scipy.special import kl_div
import time
import pickle as pkl
import matplotlib.ticker as mtick

from functools import partial

In [2]:
loc_activityClockData = '/media/ak/My Passport/ActivityClockData'
loc_ProcessedDataForExperiments = '/media/ak/My Passport1/ExperimentData/ProcessedDataForExperiments'
list_activityClockData = os.listdir('/media/ak/My Passport/ActivityClockData')
list_ProcessedDataForExperiments = os.listdir('/media/ak/My Passport1/ExperimentData/ProcessedDataForExperiments')

In [3]:


# Lets create a function that shifted your dataframe first before calling the corr().
# Let us break down what we hope to accomplish, and then translate that into code.
# For each hour (row) and for a given feature (column) I would like to find the value
# for that feature N hours prior.
# For each value of N (1-6 in our case) I want to make a new column
# for that feature representing the Nth prior hour's measurement.

elements = '/media/ak/Elements/OrderBookProcessedData'
laCie = '/media/ak/LaCie'
experimentsLocation = '/media/ak/T7/June4th2022Experiments'
bars =['volume_bar', 'calendar_bar', 'usd_volume_bar', 'tick_bar']
# plt.style.use(os.path.join('/home/ak/.config/matplotlib','l3.mplstyle'))
figures_destination = '/home/ak/Documents/Research/Papers/figures/'

jb_dict = dict()
standarised_returns = defaultdict(dict)
dataFolder = laCie
quotes = [f for f in os.listdir(dataFolder) if str('_quotes') in f]
trades = [f for f in os.listdir(dataFolder) if str('_trades') in f]

In [4]:
quotes

['YM1_Comdty_quotes',
 'VXX_US_Equity_quotes',
 'XM1_Comdty_quotes',
 'TY1_Comdty_quotes',
 'US1_Comdty_quotes']

In [5]:
# XM: australian 10yr
# YM: australian 3yr
# VXX: VIX Short-Term Futures
# US1: US 30-Year T-Bond Futures
# TY1: US 10-Year T-Note Futures


In [6]:
xm1_data = '/media/ak/OS/Data/FuturesDataSemiProcessed/XM1'
xm1_files = os.listdir(xm1_data)

In [7]:
file = os.path.join(xm1_data, xm1_files[0])
file

'/media/ak/OS/Data/FuturesDataSemiProcessed/XM1/20180416.pkl'

In [8]:
def_dict = pd.read_pickle(file)
date =list(def_dict.keys())

In [9]:
def_dict[date[0]]['tick']

,BidSize_open,BidSize_high,BidSize_low,BidSize_close,QuoteTime_open,QuoteTime_high,QuoteTime_low,QuoteTime_close,type_x_open,type_x_high,type_x_low,type_x_close,BestBid_open,BestBid_high,BestBid_low,BestBid_close,TradeTime_open,TradeTime_high,TradeTime_low,TradeTime_close,AskSize_open,AskSize_high,AskSize_low,AskSize_close,time_y_open,time_y_high,time_y_low,time_y_close,type_y_open,type_y_high,type_y_low,type_y_close,BestAsk_open,BestAsk_high,BestAsk_low,BestAsk_close,TradeSize_open,TradeSize_high,TradeSize_low,TradeSize_close,TradePrice_open,TradePrice_high,TradePrice_low,TradePrice_close,TimeStamp_open,TimeStamp_high,TimeStamp_low,TimeStamp_close,TradeVolume_open,TradeVolume_high,TradeVolume_low,TradeVolume_close,total_traded_volume_open,total_traded_volume_high,total_traded_volume_low,total_traded_volume_close,milliSeconds_open,milliSeconds_high,milliSeconds_low,milliSeconds_close,dollar_traded_volume_open,dollar_traded_volume_high,dollar_traded_volume_low,dollar_traded_volume_close,timeStampIdx_open,timeStampIdx_high,timeStampIdx_low,timeStampIdx_close,micro_price_open,micro_price_high,micro_price_low,micro_price_close,type_open,type_high,type_low,type_close
tick_bucket,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,1812,2031,1,1034,2018-06-07 00:00:19,2018-06-07 00:02:00,2018-06-07 00:00:19,2018-06-07 00:02:00,BID,BID,BID,BID,97.155,97.160,97.155,97.155,2018-06-07 00:00:19,2018-06-07 00:02:00,2018-06-07 00:00:19,2018-06-07 00:02:00,166.0,959.0,166.0,959.0,2018-06-07 00:00:19,2018-06-07 00:02:00,2018-06-07 00:00:19,2018-06-07 00:02:00,ASK,ASK,ASK,ASK,97.160,97.160,97.160,97.160,2.0,27.0,1.0,1.0,97.160,97.160,97.155,97.155,00:00:19,00:02:00,00:00:19,00:02:00,2.0,27.0,1.0,1.0,2.0,27.0,1.0,1.0,19000,120000,19000,120000,194.320,2623.185,97.155,97.155,2018-06-07 00:00:19,2018-06-07 00:02:00,2018-06-07 00:00:19,2018-06-07 00:02:00,97.155420,97.160000,97.155378,97.157406,NaN,NaN,NaN,NaN
2.0,1074,1159,1025,1076,2018-06-07 00:02:00,2018-06-07 00:04:13,2018-06-07 00:02:00,2018-06-07 00:04:13,BID,BID,BID,BID,97.155,97.155,97.155,97.155,2018-06-07 00:02:00,2018-06-07 00:04:13,2018-06-07 00:02:00,2018-06-07 00:04:13,959.0,1264.0,958.0,1264.0,2018-06-07 00:02:00,2018-06-07 00:04:13,2018-06-07 00:02:00,2018-06-07 00:04:13,ASK,ASK,ASK,ASK,97.160,97.160,97.160,97.160,1.0,4.0,1.0,1.0,97.155,97.160,97.155,97.155,00:02:00,00:04:13,00:02:00,00:04:13,1.0,4.0,1.0,1.0,1.0,4.0,1.0,1.0,120000,253000,120000,253000,97.155,388.640,97.155,97.155,2018-06-07 00:02:00,2018-06-07 00:04:13,2018-06-07 00:02:00,2018-06-07 00:04:13,97.157359,97.157761,97.157267,97.157701,NaN,NaN,NaN,NaN
3.0,1075,1075,304,595,2018-06-07 00:04:13,2018-06-07 00:05:09,2018-06-07 00:04:13,2018-06-07 00:05:09,BID,BID,BID,BID,97.155,97.155,97.155,97.155,2018-06-07 00:04:13,2018-06-07 00:05:09,2018-06-07 00:04:13,2018-06-07 00:05:09,1264.0,1590.0,1260.0,1429.0,2018-06-07 00:04:13,2018-06-07 00:05:09,2018-06-07 00:04:13,2018-06-07 00:05:09,ASK,ASK,ASK,ASK,97.160,97.160,97.160,97.160,1.0,12.0,1.0,1.0,97.155,97.160,97.155,97.155,00:04:13,00:05:09,00:04:13,00:05:09,1.0,12.0,1.0,1.0,1.0,12.0,1.0,1.0,253000,309000,253000,309000,97.155,1165.860,97.155,97.155,2018-06-07 00:04:13,2018-06-07 00:05:09,2018-06-07 00:04:13,2018-06-07 00:05:09,97.157702,97.159123,97.157702,97.158530,NaN,NaN,NaN,NaN
4.0,587,1773,1,1280,2018-06-07 00:05:09,2018-06-07 00:05:42,2018-06-07 00:05:09,2018-06-07 00:05:42,BID,BID,BID,BID,97.155,97.155,97.150,97.150,2018-06-07 00:05:09,2018-06-07 00:05:42,2018-06-07 00:05:09,2018-06-07 00:05:42,1429.0,1770.0,270.0,631.0,2018-06-07 00:05:09,2018-06-07 00:05:42,2018-06-07 00:05:09,2018-06-07 00:05:42,ASK,ASK,ASK,ASK,97.160,97.160,97.155,97.155,1.0,79.0,1.0,53.0,97.155,97.160,97.150,97.150,00:05:09,00:05:42,00:05:09,00:05:42,1.0,79.0,1.0,53.0,1.0,79.0,1.0,53.0,309000,342000,309000,342000,97.155,7675.245,97.155,5148.950,2018-06-07 00:05:09,2018-06-07 00:05:42,2018-06-07 00:05:09,2018-06-07 00:05:42,97.158544,97.159133,97.150661,97.151651,NaN,NaN,